In [3]:
import requests
import folium
import pandas as pd
import csv
import requests
import time
from datetime import datetime
import sys
import os

In [4]:
# Add the path to your project
sys.path.append('C:/Users/herde/OneDrive - The Hong Kong Polytechnic University/PhD/PhD Research/Research Objectives/Objective 4/Py_Code/netatmo_data')

In [17]:
client_id = "66c36f135947dd83b00887bb"
client_secret = "xqM6Cd5MSMhlCd87XKM5Wn87P"
refresh_token = "6639b0eb66bcd29b9809e8f4|9386ef9ff8b36f95ca546bc002654578"
lat_ne = '-26.083963'
lon_ne = '28.14711'
lat_sw = '-26.263'
lon_sw = '27.92019'
csv_file = "JB.csv"
start_date_stamp = '20150101'
start_time_stamp = '0000'
end_date_stamp = '20201231'
end_time_stamp = '2300'


In [18]:
from data_fetcher import get_access_token, fetch_station_ids, fetch_temperature_data

In [7]:
access_token = get_access_token(client_id, client_secret, refresh_token)

In [20]:
print(access_token)

6639b0eb66bcd29b9809e8f4|4b94a264b0d445f71c5441cdb9c2e640


In [21]:
access_token = get_access_token(client_id, client_secret, refresh_token)
fetch_temperature_data(access_token, csv_file, start_date_stamp, start_time_stamp, end_date_stamp, end_time_stamp)


Error: 400 {"error":"invalid_grant"}


TypeError: fetch_temperature_data() missing 2 required positional arguments: 'end_date_stamp' and 'end_time_stamp'

In [3]:
def load_device_and_module_ids_from_csv(csv_file):
    device_module_ids = []
    with open(csv_file, mode='r', newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)  # Skip the header row
        for row in reader:
            device_id = row[0]
            module_id = row[1].strip("[]'")  # Remove square brackets and single quotes
            device_module_ids.append((device_id, module_id))
    return device_module_ids

In [7]:
device_module_ids = load_device_and_module_ids_from_csv(csv_file)

In [8]:
def get_historical_measurements(access_token, device_id, module_id, scale, types, date_begin, date_end, limit=1024):
    url = "https://api.netatmo.net/api/getmeasure"
    
    params = {
        'access_token': access_token,
        'device_id': device_id,
        'module_id': module_id,
        'scale': scale,
        'type': types,
        'date_begin': date_begin,
        'date_end': date_end,
        'limit': limit
    }

    response = requests.get(url, params=params)

    if response.status_code == 200:
        return response.json()
    else:
        return None

In [9]:
def save_measurements_to_csv(measurements, device_id, module_id):
    # Remove ":" from device and module IDs for filename
    device_id_filename = device_id.replace(":", "")
    module_id_filename = module_id.replace(":", "")
    
    filename = f'{device_id_filename}_{module_id_filename}_measurements.csv'
    
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = ['acquisition_time', 'temperature', 'humidity', 'pressure']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Skip writing header if file is not empty
        if csvfile.tell() == 0:
            writer.writeheader()

        acquisition_time = measurements['body'][0]['beg_time']
        step_time = 86400
        # step_time = measurements['body'][0]['step_time']
        for entry in measurements['body']:
            for value in entry['value']:
                row = {
                    'acquisition_time': datetime.utcfromtimestamp(acquisition_time).strftime('%Y%m%d%H%M'),
                    'temperature': value[0],
                    'humidity': value[1],
                    'pressure': value[2]
                }
                writer.writerow(row)
                acquisition_time += step_time

In [10]:
def get_historical_measurements_batch(access_token, device_id, module_id, scale, types, date_begin, date_end, limit=1024):
    all_measurements = []

    while date_begin < date_end:
        measurements = get_historical_measurements(access_token, device_id, module_id, scale, types, date_begin, date_end, limit)
        if measurements is None or "body" not in measurements or not measurements["body"]:
            # No more data available or an error occurred, stop the loop
            break

        # Extract the last timestamp from the received data
        last_timestamp = measurements["body"][-1]["beg_time"]
        
        # Append the measurements to the list
        all_measurements.extend(measurements["body"])

        # Update date_begin to fetch the next batch of data
        date_begin = last_timestamp + 86400
        # date_begin = last_timestamp + measurements["body"][-1]["step_time"]

        # Save measurements to CSV file
        save_measurements_to_csv(measurements, device_id, module_id)

        # Introduce a delay to avoid hitting rate limits
        time.sleep(1)

    return all_measurements


In [11]:
def convert_to_unix_timestamp(date_string, time_string):
    try:
        # Parse the date and time strings
        date_obj = datetime.strptime(date_string, '%Y%m%d')
        time_obj = datetime.strptime(time_string, '%H%M')
        
        # Combine date and time
        datetime_obj = datetime(date_obj.year, date_obj.month, date_obj.day, time_obj.hour, time_obj.minute)
        
        # Convert to UNIX timestamp
        timestamp = int(datetime_obj.timestamp())
        return timestamp
    except ValueError:
        print("Error: Invalid input format. Please provide date in 'yyyymmdd' and time in 'hhmm' format.")
        return None

In [12]:
access_token = get_access_token(client_id, client_secret, refresh_token)

In [13]:
for device_id, module_id in device_module_ids:
    scale = "1day"  # Change according to your requirement
    types = "Temperature,Humidity,Pressure"  # Add or remove types as needed
    date_begin = convert_to_unix_timestamp(start_date_stamp, start_time_stamp)  # Start timestamp
    date_end = convert_to_unix_timestamp(end_date_stamp, end_time_stamp)  # End timestamp, set to "last" to retrieve only the last measurement
    limit = 1024  # Default limit

    # Fetch historical measurements for the current device ID and module ID pair
    measurements = get_historical_measurements_batch(access_token, device_id, module_id, scale, types, date_begin, date_end, limit)
    print(f'--------data for {device_id, module_id} completely downloaded----------')

processing_time_seconds = time.time() - start_time
processing_time_formatted = format_time(processing_time_seconds)

print("Data for all devices downloaded successfully.", f"Processing time: {processing_time_formatted}")


--------data for ('70:ee:50:06:8f:36', '02:00:00:06:9d:c0') completely downloaded----------
--------data for ('70:ee:50:1b:2a:2a', '02:00:00:1b:0c:16') completely downloaded----------
--------data for ('70:ee:50:17:d3:e0', '02:00:00:17:5e:00') completely downloaded----------
--------data for ('70:ee:50:6b:6c:26', '02:00:00:6b:2f:f6') completely downloaded----------
--------data for ('70:ee:50:6b:73:18', '02:00:00:6b:35:5e') completely downloaded----------
--------data for ('70:ee:50:64:02:5a', '02:00:00:63:fc:0c') completely downloaded----------
--------data for ('70:ee:50:7f:9e:64', '02:00:00:7f:8d:32') completely downloaded----------
--------data for ('70:ee:50:83:f3:fc', '02:00:00:83:f5:42') completely downloaded----------
--------data for ('70:ee:50:05:e1:e6', '02:00:00:12:f7:8e') completely downloaded----------
--------data for ('70:ee:50:65:11:fa', '02:00:00:65:31:0c') completely downloaded----------
--------data for ('70:ee:50:7a:70:f0', '02:00:00:7a:b0:02') completely downloade

NameError: name 'start_time' is not defined